In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import boston_housing
from keras import models, layers, losses, optimizers, metrics
from keras.utils.np_utils import to_categorical as to_cat

## 数据预处理

In [2]:
(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

In [3]:
train_data.shape

(404, 13)

In [5]:
train_targets.shape

(404,)

In [6]:
test_data.shape

(102, 13)

In [7]:
test_targets.shape

(102,)

In [8]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [11]:
train_targets[:9]

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6])

- 数据量内部差异比较大,所以要做一些小小的处理

In [13]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

In [14]:
train_data = train_data - mean
train_data = train_data / std

test_data = test_data - mean
test_data = test_data / std

In [15]:
train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [16]:
test_data[0]

array([ 1.55369355, -0.48361547,  1.0283258 , -0.25683275,  1.03838067,
        0.23545815,  1.11048828, -0.93976936,  1.67588577,  1.5652875 ,
        0.78447637, -3.48459553,  2.25092074])

## Build your nn

In [17]:
def nn():
    model = models.Sequential()
    model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizers.RMSprop(),loss=losses.mse,metrics=[metrics.mae])
    return model

### K折验证

In [18]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

In [20]:
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples], train_data[(i+1) * num_val_samples:]],axis=0
    )
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples], train_targets[(i+1) * num_val_samples:]],axis=0
    )
    
    model = nn()
    model.fit(partial_train_data,
              partial_train_targets,
              epochs=num_epochs,
              batch_size=1,
             verbose=0)
    val_mse,val_mae = model.evaluate(val_data,val_targets,verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [21]:
all_scores

[1.9942642450332642,
 2.6084344387054443,
 3.1471433639526367,
 2.8086795806884766]

In [23]:
all_scores.mean

AttributeError: 'list' object has no attribute 'mean'